In [ ]:
!pip install minicons torch pandas

In [ ]:
!pip install --upgrade transformers

In [ ]:
import torch
from minicons import scorer
import pandas as pd
from google.colab import files
from IPython.display import FileLink
from tqdm import tqdm  # 导入 tqdm 库

# ===== 核心函数 =====
def fill_classifiers_and_compute(csv_path, model_name="Qwen/Qwen3-4B", all_classifiers=None):
    """
    处理包含id/tagged/classifier列的CSV文件，并计算每个量词填充后的句子的对数概率
    """
    # 读取CSV文件（尝试逗号分隔）
    try:
        df = pd.read_csv(csv_path, sep=',', encoding='utf-8')
    except:
        # 尝试其他常见编码
        try:
            df = pd.read_csv(csv_path, sep=',', encoding='gbk')
        except Exception as e:
            raise ValueError(f"文件读取失败，请检查分隔符和编码。错误: {str(e)}")

    # 检查必要列
    required_columns = ['id', 'tagged', 'classifier']
    if not all(col in df.columns for col in required_columns):
        raise ValueError(f"文件必须包含以下列: {required_columns}\n当前列名: {list(df.columns)}")

    # 初始化模型
    print(f"正在加载模型: {model_name}...")
    model = scorer.IncrementalLMScorer(
        model_name,
        device="cuda" if torch.cuda.is_available() else "cpu"
    )

    # 保留原始正确的量词对应的对数概率
    df['complete_sentence'] = df.apply(
        lambda row: str(row['tagged']).replace('_', str(row['classifier'])),
        axis=1
    )
    print("正在计算原始句子的概率（共{}条）...".format(len(df)))
    batch_size = 32
    log_probs = []
    for i in range(0, len(df), batch_size):
        batch = df['complete_sentence'].iloc[i:i+batch_size].tolist()
        log_probs.extend(model.sequence_score(batch, reduction=lambda x: x.mean(0).item()))
    df['correct classifier\'s log proba'] = log_probs

    # 遍历所有量词，计算每个量词填充后的句子的对数概率
    for classifier, classifier_id in all_classifiers.items():
        df[f'log_prob_{classifier}'] = df.apply(
            lambda row: model.sequence_score([str(row['tagged']).replace('_', classifier)], reduction=lambda x: x.mean(0).item())[0],
            axis=1
        )

    # 重命名列，使其更直观
    for classifier, classifier_id in all_classifiers.items():
        df.rename(columns={f'log_prob_{classifier}': f'{classifier} ({classifier_id})'}, inplace=True)

    return df

# ===== 主函数 =====
def main():
    # --- 参数配置 ---
    input_file = "newtest.csv"               # 输入文件名
    model_name = "Qwen/Qwen3-4B"  # 模型名称
    output_file = "output_results.tsv"       # 输出文件名

    # --- 文件上传 ---
    print("请上传TSV/CSV文件（需包含id/tagged/classifier列）：")
    uploaded = files.upload()
    if not uploaded:
        print("错误：未检测到文件上传！")
        return

    # 获取实际上传的文件名
    actual_filename = list(uploaded.keys())[0]

    # --- 定义所有量词 ---
    all_classifiers = {
        '对': 0, '个': 1, '次': 2, '件': 3, '名': 4, '首': 5, '场': 6, '包': 7, '份': 8, '家': 9, '条': 10, '位': 11, '块': 12, '张': 13, '句': 14,
        '种': 15, '篇': 16, '类': 17, '点': 18, '滴': 19, '颗': 20, '趟': 21, '道': 22, '部': 23, '款': 24, '集': 25, '堆': 26, '瓶': 27, '只': 28,
        '盒': 29, '套': 30, '根': 31, '双': 32, '班': 33, '门': 34, '所': 35, '粒': 36, '口': 37, '座': 38, '片': 39, '等': 40, '些': 41, '番': 42,
        '顿': 43, '辆': 44, '把': 45, '杯': 46, '盆': 47, '步': 48, '群': 49, '起': 50, '回': 51, '级': 52, '碗': 53, '股': 54, '段': 55, '遍': 56,
        '号': 57, '代': 58, '阵': 59, '本': 60, '朵': 61, '波': 62, '副': 63, '章': 64, '匹': 65, '丝': 66, '分': 67, '伙': 68, '盘': 69, '声': 70,
        '排': 71, '项': 72, '轮': 73, '具': 74, '棵': 75, '顶': 76, '身': 77, '支': 78, '头': 79, '届': 80, '组': 81, '任': 82, '页': 83, '笔': 84,
        '卷': 85, '串': 86, '锅': 87, '袋': 88, '味': 89, '下': 90, '批': 91, '间': 92, '队': 93, '封': 94, '样': 95, '宗': 96, '层': 97, '圈': 98,
        '台': 99, '节': 100, '发': 101, '枚': 102, '幕': 103, '扇': 104, '人次': 105, '餐': 106, '枝': 107, '脸': 108, '剂': 109, '局': 110,
        '株': 111, '束': 112, '缸': 113, '箱': 114, '桩': 115, '季': 116, '则': 117, '通': 118, '系列': 119, '例': 120, '尊': 121, '码': 122,
        '路': 123, '般': 124, '楼': 125, '尾': 126, '列': 127, '曲': 128, '行': 129, '杆': 130, '袭': 131, '桌': 132, '线': 133, '手': 134, '团': 135,
        '艘': 136, '架': 137, '肚子': 138, '面': 139, '世': 140, '处': 141, '栋': 142, '盏': 143, '壶': 144, '幅': 145, '户': 146, '幢': 147,
        '桶': 148, '刀': 149, '堂': 150, '笼': 151, '环': 152, '册': 153, '罐': 154, '辈': 155, '拨': 156, '版': 157, '员': 158, '管': 159,
        '席': 160, '记': 161, '遭': 162, '棒': 163, '帖': 164, '档子': 165, '缕': 166, '拳': 167, '帮': 168, '族': 169, '堵': 170, '眼': 171
    }

    # --- 处理数据 ---
    try:
        df = pd.read_csv(actual_filename, sep=',', encoding='utf-8')
        required_columns = ['id', 'tagged', 'classifier']
        if not all(col in df.columns for col in required_columns):
            raise ValueError(f"文件必须包含以下列: {required_columns}\n当前列名: {list(df.columns)}")

        print(f"正在加载模型: {model_name}...")
        model = scorer.IncrementalLMScorer(
            model_name,
            device="cuda" if torch.cuda.is_available() else "cpu"
        )

        # 初始化进度条
        progress_bar = tqdm(total=len(df), desc="处理进度")

        for index, row in df.iterrows():
            complete_sentence = str(row['tagged']).replace('_', str(row['classifier']))
            log_probs = model.sequence_score([complete_sentence], reduction=lambda x: x.mean(0).item())
            df.at[index, 'complete_sentence'] = complete_sentence
            df.at[index, 'correct classifier\'s log proba'] = log_probs[0]

            for classifier, classifier_id in all_classifiers.items():
                filled_sentence = str(row['tagged']).replace('_', classifier)
                log_prob = model.sequence_score([filled_sentence], reduction=lambda x: x.mean(0).item())[0]
                df.at[index, f'{classifier} ({classifier_id})'] = log_prob

            # 更新进度条并打印当前结果
            progress_bar.update(1)
            print(f"处理完成: {row['id']} - {complete_sentence}")
            print(df.loc[index])

        progress_bar.close()

    except Exception as e:
        print(f"\n处理出错: {str(e)}")
        print("尝试解决方案：")
        print("1. 确保文件是制表符分隔的TSV或CSV")
        print("2. 检查文件是否包含id/tagged/classifier三列")
        print("3. 当前文件前5行预览：")
        print(pd.read_csv(actual_filename, sep='\t', nrows=5).head())
        return

    # --- 显示结果 ---
    print("\n处理完成！示例结果：")
    print(df.head(3))

    # --- 保存结果 ---
    df.to_csv(output_file, index=False, sep='\t', encoding='utf-8')
    print(f"\n结果已保存到: {output_file}")

    # --- 提供下载 ---
    display(FileLink(output_file))

if __name__ == "__main__":
    main()

请上传TSV/CSV文件（需包含id/tagged/classifier列）：


Saving newtest.csv to newtest (2).csv
正在加载模型: Qwen/Qwen3-4B...


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

处理进度:   0%|          | 0/1937 [00:00<?, ?it/s]<ipython-input-2-62280fa6de66>:120: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df.at[index, f'{classifier} ({classifier_id})'] = log_prob
<ipython-input-2-62280fa6de66>:120: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df.at[index, f'{classifier} ({classifier_id})'] = log_prob
<ipython-input-2-62280fa6de66>:120: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all co

处理完成: LWC_3399773837354918_02 - 无意 间 转到 一 个 摩托车 网站 , 看 着 这些 曾经 情迷 到 疯狂 的 机器 。
id                                                      LWC_3399773837354918_02
tagged                            无意 间 转到 一 _ 摩托车 网站 , 看 着 这些 曾经 情迷 到 疯狂 的 机器 。
classifier                                                                    个
complete_sentence                 无意 间 转到 一 个 摩托车 网站 , 看 着 这些 曾经 情迷 到 疯狂 的 机器 。
correct classifier's log proba                                        -2.675769
                                                      ...                      
拳 (167)                                                               -3.207872
帮 (168)                                                               -2.930568
族 (169)                                                               -2.997057
堵 (170)                                                               -3.116459
眼 (171)                                                               -2.792772
Name: 0, Length: 177, dtype: object


...

处理完成: LWC_3403421745245856_01 - 今天 早晨 多亏 烧 锅炉 大爷 没烧 , 不 给 我 冻醒 都 赶 不 上 回家 的 2 路 汽车
id                                                          LWC_3403421745245856_01
tagged                            今天 早晨 多亏 烧 锅炉 大爷 没烧 , 不 给 我 冻醒 都 赶 不 上 回家 的 2 ...
classifier                                                                        路
complete_sentence                 今天 早晨 多亏 烧 锅炉 大爷 没烧 , 不 给 我 冻醒 都 赶 不 上 回家 的 2 ...
correct classifier's log proba                                            -3.603765
                                                        ...                        
拳 (167)                                                                   -4.026862
帮 (168)                                                                   -3.882375
族 (169)                                                                   -3.905175
堵 (170)                                                                   -3.882795
眼 (171)                                                                    -3

处理进度:  69%|██████▉   | 1343/1937 [3:47:11<1:38:44,  9.97s/it]

处理完成: LWC_3401894385205685_01 - 原来 Think Different 这 句 话 是 Lee Clow 和 他 的 团队 策划 出来 重塑 Apple 形象 的
id                                                          LWC_3401894385205685_01
tagged                            原来 Think Different 这 _ 话 是 Lee Clow 和 他 的 团队 策...
classifier                                                                        句
complete_sentence                 原来 Think Different 这 句 话 是 Lee Clow 和 他 的 团队 策...
correct classifier's log proba                                            -3.450148
                                                        ...                        
拳 (167)                                                                    -4.05634
帮 (168)                                                                   -4.173481
族 (169)                                                                   -4.112747
堵 (170)                                                                    -3.91627
眼 (171)                                                        

处理进度:  69%|██████▉   | 1344/1937 [3:47:21<1:37:30,  9.87s/it]

处理完成: LWC_3405086212882970_02 - 大年 初一 的 , 打 了 三 个 小时 的 球 。
id                                   LWC_3405086212882970_02
tagged                            大年 初一 的 , 打 了 三 _ 小时 的 球 。
classifier                                                 个
complete_sentence                 大年 初一 的 , 打 了 三 个 小时 的 球 。
correct classifier's log proba                     -3.637589
                                             ...            
拳 (167)                                            -4.105939
帮 (168)                                            -4.015586
族 (169)                                              -4.5239
堵 (170)                                            -4.210741
眼 (171)                                            -4.318838
Name: 1343, Length: 177, dtype: object
